<a href="https://colab.research.google.com/github/LarrySnyder/stockpyl/blob/master/notebooks/A_Quick_Tour_of_Stockpyl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Quick Tour of `stockpyl`

This notebook gives a quick tour of the `stockpyl` Python package for inventory optimization and simulation. This notebook is essentially an interactive version of the overview given on the main page of the [documentation](https://stockpyl.readthedocs.io/en/latest/index.html).

For more info:
* `stockpyl` package on [PyPI](https://pypi.org/project/stockpyl/)
* `stockpyl` documentation on [Read the Docs](https://stockpyl.readthedocs.io/en/latest/index.html)
* `stockpyl` source code on [GitHub](https://github.com/LarrySnyder/stockpyl)

Stockpyl is a Python package for inventory optimization and simulation. It implements classical single-node inventory models like the economic order quantity (EOQ), newsvendor, and Wagner-Whitin problems. It also contains algorithms for multi-echelon inventory optimization (MEIO) under both stochastic-service model (SSM) and guaranteed-service model (GSM) assumptions. And, it has extensive features for simulating multi-echelon inventory systems.



---
> The notation and references (equations, sections, examples, etc.) used below refer to Snyder and Shen, *Fundamentals of Supply Chain Theory* (*FoSCT*), 2nd edition (2019).
---

### Installation

To install `stockpyl`, use `pip`:

In [ ]:
!pip install stockpyl

Import the package as usual:

In [ ]:
import stockpyl

### Single-Echelon Inventory Optimization

Solve the EOQ problem with a fixed cost of 8, a holding cost of 0.225, and a demand rate of 1300 (Example 3.1 in *FoSCT*):

In [ ]:
from stockpyl.eoq import economic_order_quantity
Q, cost = economic_order_quantity(fixed_cost=8, holding_cost=0.225, demand_rate=1300)
print(f"Q = {Q}, cost = {cost}")


Or the newsvendor problem with a holding cost of 0.18, a stockout cost of 0.70, and demand that is normally distributed with mean 50 and standard deviation 8 (Example 4.3 in *FoSCT*):

In [ ]:
from stockpyl.newsvendor import newsvendor_normal
S, cost = newsvendor_normal(holding_cost=0.18, stockout_cost=0.70, demand_mean=50, demand_sd=8)
print(f"S = {S}, cost = {cost}")


---
> Note that most functions in Stockpyl use longer, more descriptive parameter names (`holding_cost`, `fixed_cost`, etc.) rather than the shorter notation assigned to them in textbooks and articles (`h`, `K`).
---

Stockpyl can solve the Wagner-Whitin model using dynamic programming:

In [ ]:
from stockpyl.wagner_whitin import wagner_whitin
T = 4
h = 2
K = 500
d = [90, 120, 80, 70]
Q, cost, theta, s = wagner_whitin(T, h, K, d)
print(f"Optimal order quantities: Q = {Q}")
print(f"Optimal cost: cost = {cost}")
print(f"Cost-to-go values: theta = {theta}")
print(f"Optimal next period to order in: s = {s}")

And finite-horizon stochastic inventory problems:

In [ ]:
from stockpyl.finite_horizon import finite_horizon_dp
T = 5
h = 1
p = 20
h_terminal = 1
p_terminal = 20
c = 2
K = 50
mu = 100
sigma = 20
s, S, cost, _, _, _ = finite_horizon_dp(T, h, p, h_terminal, p_terminal, c, K, mu, sigma)
print(f"Reorder points: s = {s}")
print(f"Order-up-to levels: S = {S}")

### Multi-Echelon Inventory Optimization

Stockpyl includes an implementation of the Clark and Scarf (1960) algorithm for stochastic serial systems (more precisely, Chen-Zheng’s (1994) reworking of it):

In [ ]:
from stockpyl.supply_chain_network import serial_system
from stockpyl.ssm_serial import optimize_base_stock_levels
# Build network.
network = serial_system(
    num_nodes=3,
    node_order_in_system=[3, 2, 1],
    echelon_holding_cost=[4, 3, 1],
    local_holding_cost=[4, 7, 8],
    shipment_lead_time=[1, 1, 2],
    stockout_cost=40,
    demand_type='N',
    mean=10,
    standard_deviation=2
)
# Optimize echelon base-stock levels.
S_star, C_star = optimize_base_stock_levels(network=network)
print(f"Optimal echelon base-stock levels: S_star = {S_star}")
print(f"Optimal expected cost per period: C_star = {C_star}")


It can also optimize guaranteed-service models (GSM). For example, it implements Graves and Willem's (2003) dynamic programming algorithm for tree systems:

In [ ]:
from stockpyl.instances import load_instance
from stockpyl.gsm_tree import optimize_committed_service_times
tree = load_instance("example_6_5")
opt_cst, opt_cost = optimize_committed_service_times(tree)
print(f"Optimal committed services times: opt_cst = {opt_cst}")
print(f"Optimal cost: opt_cost = {opt_cost}")

### Simulation

Stockpyl has extensive features for simulating multi-echelon inventory systems. Below, we simulate the same serial system that we solved using Chen and Zheng's algorithm above, obtaining an average cost per period that is similar to what the theoretical model predicted.

In [ ]:
from stockpyl.supply_chain_network import echelon_to_local_base_stock_levels
from stockpyl.sim import simulation
from stockpyl.policy import Policy
# Convert to local base-stock levels and set nodes' inventory policies.
S_star_local = echelon_to_local_base_stock_levels(network, S_star)
for n in network.nodes:
    n.inventory_policy = Policy(type='BS', base_stock_level=S_star_local[n.index], node=n)
# Simulate the system.
T = 1000
total_cost = simulation(network=network, num_periods=T, rand_seed=42)
print(f"\nAverage total cost per period = {total_cost/T}")

In [ ]:
# Display the simulation results. (See https://stockpyl.readthedocs.io/en/latest/api/simulation/sim_io.html
# for a description of the column headers.)
from stockpyl.sim_io import write_results
write_results(network, num_periods=1000, periods_to_print=40, columns_to_print=['basic', 'costs'])